In [1]:
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import keras_tuner as kt
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
with open(r'.\Reviews_Dataset_Splits\X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\X_val.pkl', 'rb') as f:
    X_val = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\y_val.pkl', 'rb') as f:
    y_val = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

y_train = keras.utils.to_categorical(y_train-1, num_classes=5)
y_val = keras.utils.to_categorical(y_val-1, num_classes=5)
y_test = keras.utils.to_categorical(y_test-1, num_classes=5)

with open(r'.\embeddingMatrix_Reviews.pkl', 'rb') as f:
    embedding_matrix = pickle.load(f)

num_tokens = len(embedding_matrix) # total vocabulary +1 or length of embedding matrix
embedding_dim = 300 # dimension of the vector of a single word
MAX_REVIEW_LEN = 250 # maximum words in a review
num_classes = 5

In [3]:
def build_model(hp, min_layers, max_layers, test_optimizers, test_initializers, test_regularizer, regularizer_choice, test_learning_rate,
                test_activations, use_BatchNormalization, use_Dropout, units_min_value, units_max_value, units_step, same_units=False):
    embedding_layer = keras.layers.Embedding(
        num_tokens,
        embedding_dim,
        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
        input_length=MAX_REVIEW_LEN,
        trainable=True)
    
    model = keras.Sequential()
    model.add(embedding_layer)
    model.add(keras.layers.GlobalAveragePooling1D())

    num_layers = hp.Int('num_layers', min_layers, max_layers)

    if test_activations:
            activation = hp.Choice(f'activation', ['softplus', 'softsign', 'relu', 'tanh'])
    else:
        activation = 'relu'  # Default activation

    if test_initializers:
        kernel_initializer = hp.Choice(f'kernel_initializer', ['glorot_uniform', 'he_uniform', 'random_uniform'])
    else:
        kernel_initializer = 'he_uniform'

    if test_regularizer:
        if regularizer_choice == 'l1':
            kernel_regularizer = keras.regularizers.L1(l1=hp.Choice('l1_factor', [1e-4, 1e-2]))
        elif regularizer_choice == 'l2':
            kernel_regularizer = keras.regularizers.L2(l2=hp.Choice('l2_factor', [1e-4, 1e-2]))
        elif regularizer_choice == 'l1_l2':
            kernel_regularizer = keras.regularizers.L1L2(l1=hp.Choice('l1_l2_l1_factor', [1e-4, 1e-2]),
                                    l2=hp.Choice('l1_l2_l2_factor', [1e-4, 1e-2]))
    else:
        kernel_regularizer = None

    # Hyperparameters for the number of layers
    for i in range(num_layers):
        if same_units:
            current_units = units_min_value
        else:
            current_units = hp.Int(f'units_{i}', min_value=units_min_value, max_value=units_max_value, step=units_step)

        model.add(layers.Dense(
            units=current_units,
            activation=activation,
            kernel_initializer=kernel_initializer,
            kernel_regularizer=kernel_regularizer
        ))

        if use_BatchNormalization and hp.Boolean(f'batch_norm_{i}'):
            model.add(layers.BatchNormalization())

        if use_Dropout:
            model.add(layers.Dropout(rate=hp.Choice(f'dropout_rate_{i}', [0.0, 0.2, 0.4])))

    # Output Layer
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Optimizer
    if test_optimizers:
        optimizer = hp.Choice('optimizer', ['SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam'])
    else:
        if test_learning_rate:
            learning_rate = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4, 1e-5])
        else:
            learning_rate = 0.001
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
NUM_EPOCHS = 10
BATCH_SIZE = 128

now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
directory = f'KerasTuner_Logs/FNN_V7_3Layer_Dropout{now}'

# Callbacks
tensorboard = TensorBoard(log_dir=f'TensorBoard_Logs/FNN_V7_3Layer_Dropout{now}')
'''model_checkpoint = ModelCheckpoint(
    filepath=f"SavedModels/{directory}/best_model.h5",
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)'''
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

tuner = kt.GridSearch(
    lambda hp: build_model(hp, min_layers=3, max_layers=3, test_optimizers=False, test_initializers=False, test_regularizer=False, 
                           regularizer_choice='l1_l2', test_learning_rate=False, test_activations=False, use_BatchNormalization=False, 
                           use_Dropout=True, units_min_value=96, units_max_value=96, units_step=96, same_units=True),
    objective=kt.Objective("val_loss", direction="min"),
    max_trials=None,
    executions_per_trial=1,
    directory=directory,
    project_name='Reviews_Classification')

In [5]:
tuner.search(x=X_train,
             y=y_train,
             verbose=1,
             epochs=NUM_EPOCHS,
             batch_size=BATCH_SIZE,
             callbacks=[tensorboard, early_stopping],
             validation_data=(X_val, y_val))

Trial 27 Complete [00h 00m 26s]
val_loss: 1.1455886363983154

Best val_loss So Far: 1.0981272459030151
Total elapsed time: 00h 15m 15s


In [6]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'dropout_rate_0': 0.4,
 'dropout_rate_1': 0.2,
 'dropout_rate_2': 0.4}

In [16]:
tuner.get_best_models()[0].summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 300)          16058400  
                                                                 
 global_average_pooling1d (G  (None, 300)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 96)                28896     
                                                                 
 dropout (Dropout)           (None, 96)                0         
                                                                 
 dense_1 (Dense)             (None, 96)                9312      
                                                                 
 dropout_1 (Dropout)         (None, 96)                0         
                                                        

In [14]:
tuner.results_summary()

Results summary
Results in KerasTuner_Logs/FNN_V4_3Layer_Regularizers_2024-07-09_12-41-45\Reviews_Classification
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0001 summary
Hyperparameters:
num_layers: 3
kernel_regularizer: l2
Score: 1.609480857849121

Trial 0002 summary
Hyperparameters:
num_layers: 3
kernel_regularizer: l1_l2
Score: 1.6679480075836182

Trial 0000 summary
Hyperparameters:
num_layers: 3
kernel_regularizer: l1
Score: 1.6689815521240234


In [10]:
'''from keras.models import load_model

# Specify the path to your .h5 model file
model_path = 'SavedModels/dir_2024-07-04_16-19-08/best_model.h5'

# Load the model
s_model = load_model(model_path)
s_model.optimizer.get_config()'''